In [105]:
#Variables
R.<x> = ZZ[]
p = 2
n = 1
q = p^n
dim = 3
deg = 2 * dim

#Change p,n in variables and below in the loads to your new value of q = p^n, up to q = 9 if you want the whole file to be able to run
#can use any q if you want to run everything except the last parts after def CharpolysofsimpleAVofdim3frombounds(l,p,n):

#change 2 in the load the below to match the new value of q, so that it is load("q-weil_polynomials_dim3.sage")

load("2-weil_polynomials_dim3.sage")

#the file that has been loaded contains the characteristic polynomials of all simple abelian varieties over F_q of dimension 3
#this data has been downloaded from the LMFDB and the polynomials have been separated based on p-rank
#The LMFDB-data uses L-polynomials (which are Weil polynomials, but with the coefficients reversed)
#Hence we convert each element in the lists to Weil polynomials

def ConvertLtoWeil(LMFDBlist):
    Weilpolys = []
    for polynomials in LMFDBlist:
        d = polynomials.degree()
        newpolycoeffs = []
        for i in range(d, -1, -1):
            newpolycoeffs.append(polynomials[i])
        Weilpolys.append(R(newpolycoeffs))
    return Weilpolys

charpolydim3prank3 = ConvertLtoWeil(data_dim3_prank3)
charpolydim3prank2 = ConvertLtoWeil(data_dim3_prank2)
charpolydim3prank1 = ConvertLtoWeil(data_dim3_prank1)
charpolydim3prank0 = ConvertLtoWeil(data_dim3_prank0)
charpolydim3 = charpolydim3prank3 + charpolydim3prank2 + charpolydim3prank1 + charpolydim3prank0

#data_dim_3_prankm is the list of data of simple abelian varieties of dimension 3 over F_q with p-rank m

In [106]:
#Built-in function in Sage to generate all q-Weil polynomials of degree deg
L = R.weil_polynomials(deg, q)

In [107]:
#Generate list of all possible coefficients as in Theorem 9.2.3

def Weilpolysofdeg6frombounds(q):
    l = []

    bounda1lower = math.floor(-6 * sqrt(q)) + 1
    bounda1higher = math.ceil(6 * sqrt(q))

    for a1 in range(bounda1lower, bounda1higher):
        bounda2lower = math.floor(4 * sqrt(q) * abs(a1)) - 9 * q + 1
        bounda2higher = math.floor(a1^2 / 3 + 3 * q) + 1
        
        for a2 in range(bounda2lower, bounda2higher):
            bounda3lower1 = math.ceil(-2 * a1^3 / 27 + a1 * a2 / 3 + q * a1 - 2 * (a1^2 - 3 * a2 + 9 * q)^(3/2) / 27)
            bounda3higher1 = math.floor(-2 * a1^3 / 27 + a1 * a2 / 3 + q * a1 + 2 * (a1^2 - 3 * a2 + 9 * q)^(3/2) / 27) + 1
            
            bounda3lower2 = math.floor(-2 * q * a1 - 2 * sqrt(q) * a2 - 2 * q * sqrt(q)) + 1
            bounda3higher2 = math.ceil(-2 * q * a1 + 2 * sqrt(q) * a2 + 2 * q * sqrt(q))
            
            bounda3lower = max(bounda3lower1, bounda3lower2)
            bounda3higher = min(bounda3higher1, bounda3higher2)
            
            for a3 in range(bounda3lower, bounda3higher):
                coeff = [q^3, q^2 * a1, q * a2, a3, a2, a1, 1]
                l.append(R(coeff))

    return l

l = Weilpolysofdeg6frombounds(q)

In [108]:
#Check for differences between lists generated by the built-in function and the one generated by Theorem 9.2.3
def comparisonlists(l,L):
    Missing = []

    for e in L:
        if e not in l:
            Missing.append(e)
            print(e.factor())

    print("number of polynomials missed by the bounds:", len(Missing))

    extra = []
    for f in l:
        if f not in L:
            extra.append(f)
            print(f.factor())

    print("number of extra polynomials found by the bounds", len(extra))

    Missingreals = []

    for pols in Missing:
        if (pols(sqrt(q)) == 0) or (pols(- sqrt(q)) == 0):
            Missingreals.append(pols)

    print("number of the missing polynomials that have a real root:", len(Missingreals))

    realsinresult = []

    for realtest in l:
        if (realtest(sqrt(q)) == 0) or (realtest(- sqrt(q)) == 0):
            realsinresult.append(realtest)

    print("number of weil polynomials with real roots satisfying the bounds:", len(realsinresult))

comparisonlists(l,L)

(x^2 + 2*x + 2) * (x^2 - 2)^2
(x^2 + x + 2) * (x^2 - 2)^2
(x^2 + 2) * (x^2 - 2)^2
(x^2 - x + 2) * (x^2 - 2)^2
(x^2 - 2*x + 2) * (x^2 - 2)^2
number of polynomials missed by the bounds: 5
number of extra polynomials found by the bounds 0
number of the missing polynomials that have a real root: 5
number of weil polynomials with real roots satisfying the bounds: 0


In [111]:
#The polynomials missing are exactly the ones with a real root, as described in Theorem 9.2.2 (and 9.2.1 if q is a square)

In [109]:
#check which Weil polynomials are characteristic polynomials of simple abelian varieties over F_q as in Theorem 9.6

def CharpolysofsimpleAVofdim3frombounds(l,p,n):

    prank0 = []
    prank1 = []
    prank2 = []
    prank3 = []

    Rp = Zp(p)
    vp = Rp.valuation()
    S.<y> = Zp(p, prec = 100, type = 'capped-rel', print_mode = 'series')[]

    for polyns in l:
        if polyns.is_irreducible():
            d = polyns.degree()
            vpa3 = vp(polyns[d - 3])
            vpa2 = vp(polyns[d - 2])
            vpa1 = vp(polyns[d - 1])
        
            if vpa3 == 0:
                prank3.append(polyns)
            else:
                rootsof = polyns.roots(Rp)
                rv = []
                if not len(rootsof) == 0:
                    for roots in rootsof:
                        rv.append(vp(roots[0]))
            
                coeffsinZp = []
                for co in range(polyns.degree() + 1):
                    coeffsinZp.append(polyns[co])
                    polynsinZp = S(coeffsinZp)
            
                factorinZp = polynsinZp.factor()
                fd = []
                for fa in factorinZp:
                    fd.append(fa[0].degree())
            
                if vpa3 >= n/2 and vpa2 == 0 and n/2 not in rv:
                    prank2.append(polyns)
            
                elif vpa3 >= n and vpa2 >= n/2 and vpa1 == 0 and n/2 not in rv:
                    prank1.append(polyns)
                    
                elif vpa3 == n and vpa2 >= 2/3 * n and vpa1 >= n/3 and len(rv) == 0:
                    prank0.append(polyns)
                
                elif vpa3 >= 3/2 * n and vpa2 >= n and vpa1 >= n/2 and len(rv) == 0 and 3 not in fd:
                    prank0.append(polyns)            
            
        elif n % 3 == 0 and len(polyns.factor()) == 1:
            if polyns.factor()[0][1] == 3:
                omega = polyns.factor()[0][0][1]
                if abs(omega) < 2 * sqrt(q) and omega % q^(1/3) == 0:
                    if not omega / q^(1/3) % p == 0:
                        prank0.append(polyns)

    total = prank0 + prank1 + prank2 + prank3

    return prank3, prank2, prank1, prank0, total

prank3, prank2, prank1, prank0, total = CharpolysofsimpleAVofdim3frombounds(l,p,n)

#beneath is the comparison of characteristic polynomials of simple abelian varieties of dimension 3 over F_q with data from LMFDB

In [110]:
Missing3 = []

for m in charpolydim3prank3:
    if m not in prank3:
        Missing3.append(m)

extra3 = []

for M in prank3:
    if M not in charpolydim3prank3:
        extra3.append(M)

print("number of characteristic polynomials p-rank 3 in LMFDB:", len(data_dim3_prank3))
print("number of characteristic polynomials p-rank 3 according to code:", len(prank3))
print("characteristic polynomials in LMFDB but not in code:", Missing3)
print("characteristic polynomials in code, but not in LMFDB:", extra3)

number of characteristic polynomials p-rank 3 in LMFDB: 56
number of characteristic polynomials p-rank 3 according to code: 56
characteristic polynomials in LMFDB but not in code: []
characteristic polynomials in code, but not in LMFDB: []


In [111]:
Missing2 = []

for m in charpolydim3prank2:
    if m not in prank2:
        Missing2.append(m)

extra2 = []

for M in prank2:
    if M not in charpolydim3prank2:
        extra2.append(M)        
        
print("number of characteristic polynomials p-rank 2 in LMFDB:", len(data_dim3_prank2))
print("number of characteristic polynomials p-rank 2 according to code:", len(prank2))
print("characteristic polynomials in LMFDB but not in code:", Missing2)
print("characteristic polynomials in code, but not in LMFDB:", extra2)

number of characteristic polynomials p-rank 2 in LMFDB: 12
number of characteristic polynomials p-rank 2 according to code: 12
characteristic polynomials in LMFDB but not in code: []
characteristic polynomials in code, but not in LMFDB: []


In [112]:
Missing1 = []

for m in charpolydim3prank1:
    if m not in prank1:
        Missing1.append(m)

extra1 = []

for M in prank1:
    if M not in charpolydim3prank1:
        extra1.append(M)  
        
print("number of characteristic polynomials p-rank 1 in LMFDB:", len(data_dim3_prank1))
print("number of characteristic polynomials p-rank 1 according to code:", len(prank1))
print("characteristic polynomials in LMFDB but not in code:", Missing1)
print("characteristic polynomials in code, but not in LMFDB:", extra1)

number of characteristic polynomials p-rank 1 in LMFDB: 4
number of characteristic polynomials p-rank 1 according to code: 4
characteristic polynomials in LMFDB but not in code: []
characteristic polynomials in code, but not in LMFDB: []


In [113]:
Missing0 = []

for m in charpolydim3prank0:
    if m not in prank0:
        Missing0.append(m)

extra0 = []

for M in prank0:
    if M not in charpolydim3prank0:
        extra0.append(M)  
        
print("number of characteristic polynomials p-rank 0 in LMFDB:", len(data_dim3_prank0))
print("number of characteristic polynomials p-rank 0 according to code:", len(prank0))
print("characteristic polynomials in LMFDB but not in code:", Missing0)
print("characteristic polynomials in code, but not in LMFDB:", extra0)

number of characteristic polynomials p-rank 0 in LMFDB: 8
number of characteristic polynomials p-rank 0 according to code: 8
characteristic polynomials in LMFDB but not in code: []
characteristic polynomials in code, but not in LMFDB: []


In [114]:
Missingtotal = Missing0 + Missing1 + Missing2 + Missing3
extratotal = extra0 + extra1 + extra2 + extra3

print("total number of characteristic polynomials in LMFDB:", len(charpolydim3))
print("total number of characteristic polynomial according to code:", len(total))
print("characteristic polynomials in LMFDB but not in code:", Missingtotal)
print("characteristic polynomials in code but not in LMFDB:", extratotal)

total number of characteristic polynomials in LMFDB: 80
total number of characteristic polynomial according to code: 80
characteristic polynomials in LMFDB but not in code: []
characteristic polynomials in code but not in LMFDB: []
